In [1]:
from glob import glob
from itertools import product
from noise import add_noise
import mat73
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from training_sktime import normalizing
import numpy as np 
import random
from noise import compressed_pickle, decompress_pickle
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
INPUT_DATA_PATH = '../input-data/'
MODEL_PATH = './models/'

In [2]:
path = '../input-data/dados_robson/Sinais_Robson/'
files = glob(path + '*.mat')

In [3]:
disparo = [file for file in files if 'disparo' in file ][0]
for i, file in enumerate(files):
    if 'disparo' in file:
        disparo = file
        del files[i]
distance = range(3)
resistance = range(3)
compensation = range(4)
angle = range(19)

indexes = list(product(distance, resistance, compensation, angle))

In [4]:
def get_distance(i):
    if i[0] == 0:
        return '20'
    elif i[0] == 1:
        return '150'
    else:
        return '280'

def get_resistance(i):
    if i[1] == 0:
        return '1'
    elif i[1] == 1:
        return '50'
    else:
        return '100'

def get_compensation(i):
    if i[2] == 0:
        return '0.4'
    elif i[2] == 1:
        return '0.5'
    elif i[2] == 2:
        return '0.6'
    else:
        return '0.7'

def get_angle(i):
    return str(i[3] * 10)

In [5]:
angle_dict = {}
for i, item in enumerate(sio.loadmat(disparo)['Flt_trip']):
    value = np.where(item[:-1] != item[1:])[0][0]
    angle_dict[i] = value

In [6]:
cycle_dict = {'32': 8, '64': 4, '128': 2}

In [7]:
signals = []
for file in files:
    data = mat73.loadmat(file)
    fault_type = file.split('_')[-1].split('.')[0]
    signal = data[f'I_{fault_type}']
    final_dict_signal = {}
    # samples = [indexes[i] for i in random.sample(range(684), 94)]
    for i in indexes:
        phase = signal[:,:, i[0], i[1], i[2], i[3]]
        index = angle_dict[i[3]]
        distance = get_distance(i)
        resistance = get_resistance(i)
        compensation = get_compensation(i)
        angle = get_angle(i)
        fault = fault_type.replace('G', 'T')
        final_dict_signal = {'distance': distance, 'resistance': resistance,
                                'compensation': compensation, 'angle': angle, 'fault_type': fault}

        # Subdivide sinal com ciclos pós falta
        for c, v in cycle_dict.items():
            detected_signal = phase[index-64:index+v]
            phase_a = add_noise(detected_signal[:,0], 60)
            phase_b = add_noise(detected_signal[:,1], 60)
            phase_c = add_noise(detected_signal[:,2], 60)
            phase_z = phase_a + phase_b + phase_c
            
            final_detected = np.vstack((phase_a, phase_b, phase_c, phase_z)).flatten()
            final_dict_signal.update({f'i_cycle_{c}': final_detected})

        signals.append(final_dict_signal)

In [8]:
def format_dataframe(data):
    cols = int(data.shape[0] / 4)
    shaped_data = data.reshape((4, cols)).T
    s1 = pd.Series(shaped_data[:, 0])
    s2 = pd.Series(shaped_data[:, 1])
    s3 = pd.Series(shaped_data[:, 2])
    s4 = pd.Series(shaped_data[:, 3])
    dicio = {'A': [], 'B': [], 'C': [], 'Z': []}
    dicio['A'].append(s1)
    dicio['B'].append(s2)
    dicio['C'].append(s3)
    dicio['Z'].append(s4)
    return pd.DataFrame(dicio)

In [9]:
def open_data(signal_type, cycle_name):
    data_list = []
    target_list = []
    for d in signals:
        data_list.append(format_dataframe(d[f'{signal_type}_{cycle_name}']))
        target_list.append(d['fault_type'])
    X = pd.concat(data_list).reset_index(drop=True)
    y = np.array(target_list)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,
                                                        random_state=42, shuffle=True)
    compressed_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal_type}/{cycle_name}/' + 'X_train', X_train)
    compressed_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal_type}/{cycle_name}/' + 'y_train', y_train)
    compressed_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal_type}/{cycle_name}/' + 'X_val', X_test)
    compressed_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal_type}/{cycle_name}/' + 'y_val', y_test)
    return X_train, y_train

In [10]:
def save_folds(signal_type, cycle_name):
    X_train, y_train = open_data(signal_type, cycle_name)
    data_folds_path = INPUT_DATA_PATH + f'folds-robson-completo/{signal_type}/{cycle_name}/'
    kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    for fold, (tr, te) in enumerate(kf.split(X_train, y_train), start=1):
        X_tr, X_te = X_train.iloc[tr, :], X_train.iloc[te, :]
        y_tr, y_te = y_train[tr], y_train[te]
        compressed_pickle(data_folds_path + f'X_train_fold_{fold}', X_tr)
        compressed_pickle(data_folds_path + f'X_test_fold_{fold}', X_te)
        compressed_pickle(data_folds_path + f'y_train_fold_{fold}', y_tr)
        compressed_pickle(data_folds_path + f'y_test_fold_{fold}', y_te)

In [12]:
cycle_list = ['cycle_32', 'cycle_64', 'cycle_128']

for cycle_name in cycle_list:
    save_folds('i', cycle_name)

# Treinamento

In [14]:
from glob import glob
from itertools import product
from noise import add_noise, decompress_pickle
import mat73
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from training_sktime import normalizing, format_dataframe
from sklearn.metrics import accuracy_score
import pickle
import time
import plotly.figure_factory as ff

In [17]:
def open_folds(cycle, train_test, X_y, v_i):
    """
    Parameters:
        cycle      : which cycle, ex.: 'cycle_1' (1, 2, 4, 8, 16, 32, 64, 128...)
        train_test : if it is the train ot test set, ex: 'train' (train, test)
        X_y        : if it is the X or y set, ex.: 'X' (X, y)
        v_i        : if it is a voltage or current signal, ex.: 'i' (v, i)
    Return:
        list : each fold is in a position.
    """
    paths_robson = list(map(lambda x: x.split('.pbz2')[0],
                            glob(INPUT_DATA_PATH + f'folds-robson-completo/{v_i}/{cycle}/{X_y}_{train_test}_fold_[0-9]*.pbz2')))
    paths_robson.sort(key = lambda x: int(x.split('_')[-1]))
    data_list = []
    for path in paths_robson:
        folder_pos = int(path.split('/')[-1].split('_')[-1]) - 1
        fold = decompress_pickle(path)
        data_list.insert(folder_pos, fold)
    return data_list

def generate_title(cycle, model_name):
    title = cycle.split('_')[-1]
    if title != '1':
        title = f'{model_name.title()} e 1/{title} ciclo pós falta'
    else:
        title = f'{model_name.title()} e 1 ciclo pós falta'
    return title

def evaluating_model(model, transformation, X_test, y_test, cycle, scores, count, max_list, model_name='model', save=None):
    # Evaluating model
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    scores.append(score)
    if save and (
        len(scores) != 1 and score > scores[count] or len(scores) == 1
    ):
        pickle.dump(transformation, open(MODEL_PATH + f'novo_treino_{model_name}_{cycle}.pkl', 'wb'))
        pickle.dump(model, open(MODEL_PATH + f'novo_treino_{model_name}_classifier_{cycle}.pkl', 'wb'))
        pickle.dump(max_list, open(MODEL_PATH + f'novo_treino_{model_name}_{cycle}_max_values.pkl', 'wb'))
    return scores

def print_results(cycle, model_name, scores, end_time, start_time, save=None):
    folds_labels = [f'- Fold {i}' for i in range(1, 11)]
    f = open(f'novo_treino_{model_name}_report.txt','a') if save else save
    title = generate_title(cycle, model_name)
    print('\nAcurácia em cada fold:\n')
    for k, v in dict(zip(folds_labels, np.round(scores * 100, decimals=2))).items():
        print(f'{k:<7}: {v:^7.2f}%')
    print('\nO resulto final obtido foi:\n')
    print(f'- Média da acurácia: {np.mean(scores) * 100:.2f}%')
    print(f'- Desvio padrão da acurácia: {np.std(scores) * 100:.2f}%')
    print(f'- Tempo necessário para treinamento: {np.round(end_time - start_time, 3)} segundos')
    
def kfold(train_X, train_y, test_X, test_y, model, cycle, max_list, model_name='',
          transformation=None, save=None):
    scores = []
    s = time.time()
    for count, (X_tr, y_tr, X_te, y_te) in enumerate(zip(train_X, train_y, test_X, test_y),
                                                     start=-1):
        X_tr_norm = normalizing(X_tr, max_list)
        X_te_norm = normalizing(X_te, max_list)

        # Transforming data
        if transformation:
            X_tr_transform = transformation.transform(X_tr_norm)
            X_te_transform = transformation.transform(X_te_norm)
        else:
            X_tr_transform = X_tr_norm.copy()
            X_te_transform = X_te_norm.copy()

        model.fit(X_tr_transform, y_tr)
        scores = evaluating_model(model, transformation, X_te_transform, y_te, cycle, scores,
                                  count, max_list, model_name, save)

    e = time.time()
    final_scores = np.array(scores)
    print_results(cycle, model_name, final_scores, e, s, save)
    return np.mean(scores) * 100, np.round(e - s, 3)

def validating(X_val, y_val, model_name, cycle, max_list, save=None):
    s = time.time()
    with open(MODEL_PATH + f'novo_treino_{model_name}_classifier_{cycle}.pkl', 'rb') as f:
        best_model = pickle.load(f)
    val_score = best_model.score(X_val, y_val)
    y_pred = best_model.predict(X_val)
    e = time.time()
    f = open(f'novo_treino_{model_name}_report.txt','a') if save else save
    print(f'- Acurácia no conjunto de validação: {val_score * 100:.2f}%')
    print(f'- Tempo necessário para predição do conjunto de validação: {np.round(e - s, 3)} segundos')
    return y_pred, val_score * 100, np.round(e - s, 3)

def generate_confusion_matrix(y_val, y_pred, image_path, filename, title='', colorscale='blues',
                              width=500, height=500):
    data = {'Real':    y_val,
            'Predito': y_pred}
    df = pd.DataFrame(data, columns=['Real','Predito'])
    confusion_matrix = pd.crosstab(df['Real'], df['Predito'], rownames=['Real'],
                                   colnames=['Predito'], margins = True)
    cm = confusion_matrix.drop('All', axis=1).drop('All', axis=0)
    # Inverte rows because create_annotated_heatmap creates matrix in inverted order
    c = cm.values[::-1]
    x = list(cm.index)
    y = x[::-1]
    c_text = [[str(y) for y in x] for x in c]
    fig = ff.create_annotated_heatmap(c, x=x, y=y, annotation_text=c_text, colorscale=colorscale)
    # add title
    fig.update_layout(title_text=f'<i><b>Matriz de Confusão {title}</b></i>',
                      title_x=0.5, autosize=False, width=width, height=height,)
    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14), x=0.5, y=-0.12, showarrow=False,
                            text="Valores Preditos", xref="paper", yref="paper"))
    fig.add_annotation(dict(font=dict(color="black",size=14), x=-0.2, y=0.5, textangle=270,
                            showarrow=False, text="Valores Reais", xref="paper", yref="paper"))
    fig.write_image(image_path + filename + '.svg')

def find_max(X):
    max = np.max(X)
    min = np.abs(np.min(X))
    if max > min:
        return max
    else:
        return min

def find_max_value(X):
    # Finding max value in by phase in training set to normalize signals
    max_list = [0, 0, 0, 0]
    for k, v in {'A': 0, 'B': 1, 'C': 2, 'Z': 3}.items():
        for row in X[k]:
            max_value = find_max(row)
            if max_value > max_list[v]:
                max_list[v] = max_value
    return max_list

def training(signal, cycle, model, model_name='', transformation=None, save=None):
    X_train = decompress_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal}/{cycle}/X_train')
    X_val = decompress_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal}/{cycle}/X_val')
    y_val = decompress_pickle(INPUT_DATA_PATH + f'folds-robson-completo/{signal}/{cycle}/y_val')

    max_list = find_max_value(X_train)
    X_train_norm = normalizing(X_train, max_list)
    X_val_norm = normalizing(X_val, max_list)
        
    if transformation:
        transformation.fit(X_train_norm)
        X_val_transform = transformation.transform(X_val_norm)
    else:
        X_val_transform = X_val_norm.copy()

    # Opening the folds and saving as lists for training and testing 
    train_X = open_folds(cycle, 'train', 'X', signal)
    train_y = open_folds(cycle, 'train', 'y', signal)
    test_X = open_folds(cycle, 'test', 'X', signal)
    test_y = open_folds(cycle, 'test', 'y', signal)
    mean_acc, train_time = kfold(train_X, train_y, test_X, test_y, model, cycle, max_list, model_name, transformation, save)
    y_pred, val_acc, val_time = validating(X_val_transform, y_val, model_name, cycle, max_list, save)
    # title = generate_title(cycle, model_name)
    # generate_confusion_matrix(y_val, y_pred, 'figs_cm/new_dataset/', f'{cycle}_{model_name}', title=title)
    # print(f'Finalizado treinamento para {title}!')
    return mean_acc, val_acc, train_time, val_time

In [18]:
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket, MiniRocketMultivariate

print('\n### Treinando com 10000 features (default)', sep='')
transformation = MiniRocketMultivariate(random_state=42)
model = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
signal, model_name = 'i', 'minirocket'
for cycle in ['cycle_32']:
    print('\n---')
    c = cycle.split('_')[-1]
    if c == '1':
        title = f'\n## {c} Ciclo Pós Falta'
    else:
        title = f'\n## 1/{c} Ciclo Pós Falta'
    mean_acc, val_acc, train_time, val_time = training(signal, cycle, model, model_name, transformation, save=True)
    row = f'\n|{title.split(" ")[1]}|10000|{mean_acc:.2f}|{val_acc:.2f}|{train_time}|{val_time}|'


### Treinando com 10000 features (default)

---

Acurácia em cada fold:

- Fold 1:  46.72 %
- Fold 2:  51.09 %
- Fold 3:  52.65 %
- Fold 4:  49.54 %
- Fold 5:  51.19 %
- Fold 6:  48.99 %
- Fold 7:  47.90 %
- Fold 8:  46.80 %
- Fold 9:  47.71 %
- Fold 10:  46.98 %

O resulto final obtido foi:

- Média da acurácia: 48.96%
- Desvio padrão da acurácia: 1.99%
- Tempo necessário para treinamento: 685.281 segundos
y de validação
['AT' 'CAT' 'BT' ... 'CT' 'ABT' 'BC']
y de predição final
['AT' 'CAT' 'CA' ... 'CT' 'BC' 'BC']
- Acurácia no conjunto de validação: 48.10%
- Tempo necessário para predição do conjunto de validação: 0.576 segundos
